# Association Rule Mining in Mass-Spectrometry Data
### *Muhammad Haseeb and Muhammad Usman Tariq*
##### *CAP6778: Adv. Topics in Data Mining - Fall 2019*
##### *School of Computing and Information Sciences (SCIS)*
##### *Florida International University (FIU)*

### The MIT License

##### *Copyright 2019 Muhammad Haseeb and Muhammad Usman Tariq*

Permission is hereby granted, free of charge, to any person obtaining a copy of this software 
and associated documentation files (the "Software"), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial 
portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT 
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Required Imports

In [1]:
# Import the required libraries
import os
import re
import sys
import math
import time
import urllib
import operator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from IPython.display import clear_output

## Data Acquisition
Let's get the annotated HUMAN MS/MS spectral data libraries from <a href="https://chemdata.nist.gov/dokuwiki/doku.php?id=peptidew:cdownload" class="external">NIST.</a> Note that this can be done for any spectral library or multiple spectral libraries at once as well.

In [2]:
mouse = 'cptac2_mouse_hcd_selected.msp.tar.gz'
msp_file = './data/cptac2_mouse_hcd_selected.msp'
data_dir = os.path.expanduser("./data")

if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

if not os.path.exists(msp_file):
    if not os.path.exists(mouse):
        !wget 'ftp://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz'
    
    # Extract the tar ball into MSP file and move into the speclib folder
    !tar xvf 'cptac2_mouse_hcd_selected.msp.tar.gz'
    os.rename('./cptac2_mouse_hcd_selected.msp', msp_file)

### Data Extraction Hyperparameters

In [3]:
# Data extraction Hyperparamters
divider = 5                     # Divide every m/z by this number.
max_len = int(5000 / divider)  # Max allowed m/z value.
filter_peaks = 100              # How many peaks to keep after filtration.

In [4]:
'''To keep updating this dict for multiple files. Dont define it again.'''
pep_spec = {}

### MSP File Parser

Read the MSP file, extract the required ions and pre-prcocess spectra

In [5]:
f=open(msp_file, "r")
lines = f.readlines()
f.close()

isName = isMW = isNumPeaks = False
new = prev = 0
i = 0


while i < len(lines):
    '''
    Process each line of the file.
    '''
    line = lines[i]
    i += 1
    
    splits = line.split(':')
    
    '''Keep going through lines. Name line is peptide and
    all the lines after Num peaks are the spectrum'''
    if splits[0] == 'Name':
        split1 = splits[1]
        pep = split1.split('/')[0].lstrip(' ')
        isName = True
        
    if isName and splits[0] == 'MW':
        mass = float(splits[1]) # Not using it anywhere. It's just there.
        isMW = True
        
    if isName and isMW and splits[0] == 'Num peaks':
        '''After this line, we will start processing the spectrum.'''
        numPeaks = int(splits[1]) # Just in case.
        
        temp_spec = np.zeros(max_len)
        while (lines[i] != '\n'):
            mzline = lines[i]
            i += 1
            mzsplits = mzline.split('\t')
            # Get the m/z and intenisty value of each line.
            moz, intensity, pattern = float(mzsplits[0]), float(mzsplits[1]), mzsplits[2]

            # Filter out only the non-noisy terminal ions from the spectrum
            lbls = re.findall(r"(?<!\/)(?<![A-Z])([abcxyz][0-9]+)", pattern)

            if lbls:
                # Dividing m/z by the divider.
                temp_spec[round(moz / divider)] = intensity

        isNumPeaks = True
        
    if isName and isMW and isNumPeaks:
        '''At this point, we are done reading one spectrum.
        Place everything where it needs to go and read the next one.'''
        isName = isMW = isNumPeaks = False
        
        # This is the filter. Gets the top 100 peaks
        # and returns them in one hot encoded vector of size max_len
        top_indx = np.argpartition(temp_spec, -filter_peaks)[-filter_peaks:]
        spec = np.zeros(max_len, dtype=bool)
        spec[top_indx] = True
        
        # Creating dictionary. Key: peptide, Value: spectrum.
        # As a side effect we'll get rid of duplicates.
        if pep not in pep_spec: pep_spec[pep] = spec
        
        # Displays the progress.
        new = int((i/len(lines)) * 100)
        if new > prev:
            clear_output(wait=True)
            print(str(new) + '%')
            prev = new


99%


## Frequent Itemset Mining
Construct the market-basket dataframe.

### Frequent Itemset Hyperparameters

In [6]:
max_itemset_size = 4     # Max size of the frequent itemsets.
min_support = 0.40       # Support of 80%

### DataFrame Construction

In [7]:
# Just get the value from the dict.
buckets = list(pep_spec.values())
print(len(buckets))

# The apriori function needs a dataframe.
df_buckets = pd.DataFrame(buckets, columns=list(range(0, max_len)))

10026


In [8]:
df_buckets

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


### Apriori Algorithm
Using the association rule mining algorithms from: *Agrawal, Rakesh, and Ramakrishnan Srikant. "Fast algorithms for mining association rules." Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994*. Using implementation from the [mlxtend.apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) Python package.

In [9]:
freq_itemsets = apriori(df_buckets,
                        min_support=min_support, 
                        max_len=max_itemset_size, 
                        verbose=1,
                        low_memory=True)

Iteration: 650895 | Sampling itemset size 4


### Freq Itemsets Visualization

Now that we have mined the frequent itemsets from the dataframe, let's visualize some stats and see if we need to refine any hyperparameters and mine itemsets again.

In [10]:
# Add another column to store lengths of the frequent itemsets
freq_itemsets['length'] = freq_itemsets['itemsets'].apply(lambda x: len(x))

# Filter out the small itemsets (of length < 3)
filtered_freqITS = freq_itemsets[ (freq_itemsets['length'] > 2) &
                                        (freq_itemsets['support'] > 0.8) ]
# We found 625 filtered frequent itemsets
disp_ffITS = filtered_freqITS.reset_index().rename(columns={'index' : 'itemset#'})

filtered_freqITS = filtered_freqITS.drop(['length'], axis=1)

In [11]:
disp_ffITS

,itemset#,support,itemsets,length
0,36387,0.804608,"(352, 353, 351)",3
1,36388,0.803212,"(352, 354, 351)",3
2,36412,0.803511,"(353, 354, 351)",3
3,36413,0.800020,"(353, 355, 351)",3
4,36436,0.800319,"(354, 355, 351)",3
5,36711,0.809595,"(352, 353, 354)",3
6,36712,0.805506,"(352, 353, 355)",3
7,36713,0.803012,"(352, 353, 356)",3
8,36714,0.801217,"(352, 353, 357)",3
9,36715,0.800818,"(352, 353, 358)",3


We found about 625 frequent itemsets with support > 0.8 and length > 2. These look good. Let's mine the association rules.

## Association Rule Mining

Let's mine association rules from the obtained frequent itemsets and take a look at rules ($R = I\rightarrow j$) with high *confidence* ($Conf$) and *interest* ($Interest$):

$$
Conf(I\rightarrow j) = \frac{Support(I \cup j)}{Support(I)} \\
Interest(I \rightarrow j) = Conf(I\rightarrow j) - Pr [j]
$$
where $Pr [j]$ is the probability of the item $j$ computed as: $support(j)$

### Association Rules Hyperparameters

In [12]:
confidence = 0.80     # Association Rule Confidence
interest   = 0.50     # Association Rule Interest (leverage)

### Association Rules
Using the association rule mining algorithms from: *Agrawal, Rakesh, and Ramakrishnan Srikant. "Fast algorithms for mining association rules." Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994*. Using implementation from the [mlxtend.association_rules](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/) Python package.

In [13]:
# Get association rules based on the confidence 
rules_conf = association_rules(freq_itemsets, 
                               metric='confidence', 
                               min_threshold=confidence, 
                               support_only=False)

In [14]:
# Get association rules based on the interest
rules_conf['interest'] = rules_conf['confidence'] - rules_conf['consequent support']

### Association Rules Visualization

In [15]:
rules_conf

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,interest
0,(26),(29),0.486036,0.509675,0.403950,0.831110,1.630667,0.156229,2.903225,0.321435
1,(26),(345),0.486036,0.776980,0.405346,0.833983,1.073365,0.027706,1.343359,0.057003
2,(26),(346),0.486036,0.783363,0.409635,0.842807,1.075883,0.028892,1.378160,0.059444
3,(26),(347),0.486036,0.790844,0.415520,0.854915,1.081016,0.031141,1.441610,0.064071
4,(26),(348),0.486036,0.801217,0.419808,0.863739,1.078034,0.030388,1.458841,0.062522
5,(26),(349),0.486036,0.807500,0.423998,0.872358,1.080319,0.031523,1.508119,0.064857
6,(26),(350),0.486036,0.810792,0.428286,0.881182,1.086816,0.034212,1.592419,0.070390
7,(26),(351),0.486036,0.820068,0.431378,0.887544,1.082281,0.032796,1.600017,0.067476
8,(26),(352),0.486036,0.826252,0.435568,0.896163,1.084612,0.033979,1.673271,0.069911
9,(26),(353),0.486036,0.832535,0.439258,0.903755,1.085546,0.034615,1.739989,0.071220


# Results

In [16]:
# Let's sort the results based on the leverage (interest)
res = rules_conf.sort_values(by=['interest'], ascending=False)
res = res[res['interest']>=interest]

In [22]:
res

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,interest
2011376,"(331, 326)","(328, 329)",0.426192,0.455815,0.417115,0.978703,2.147151,0.222851,25.552780,0.522889
2011364,"(330, 326)","(328, 329)",0.426192,0.455815,0.416916,0.978235,2.146124,0.222651,25.003258,0.522421
2011388,"(332, 326)","(328, 329)",0.427987,0.455815,0.417016,0.974365,2.137633,0.221933,21.228167,0.518550
2011400,"(333, 326)","(328, 329)",0.428586,0.455815,0.416816,0.972539,2.133627,0.221461,19.816639,0.516724
1143332,"(283, 326)","(328, 329)",0.414622,0.455815,0.403052,0.972095,2.132654,0.214061,19.501531,0.516280
2011424,"(326, 335)","(328, 329)",0.429084,0.455815,0.416816,0.971409,2.131147,0.221233,19.033206,0.515594
2011412,"(334, 326)","(328, 329)",0.428985,0.455815,0.416617,0.971169,2.130623,0.221079,18.875324,0.515355
2011460,"(338, 326)","(328, 329)",0.428985,0.455815,0.416417,0.970704,2.129602,0.220880,18.575716,0.514890
2011472,"(339, 326)","(328, 329)",0.429084,0.455815,0.416417,0.970479,2.129107,0.220834,18.433735,0.514664
2011436,"(336, 326)","(328, 329)",0.429384,0.455815,0.416417,0.969803,2.127624,0.220698,18.020900,0.513988


## Interpretation of Results
So we have 64 rules of our interest (high *support, confidence* and *interest*). Let's dig deeper in these results.

In [18]:
df_buckets.columns = df_buckets.columns.map(str)
df_buckets.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '990', '991', '992', '993', '994', '995', '996', '997', '998', '999'],
      dtype='object', length=1000)

In [19]:
specs = np.array(df_buckets[(df_buckets['329'] == True) & 
                            (df_buckets['334'] == True) & 
                            (df_buckets['331'] == True) & 
                            (df_buckets['322'] == True)].reset_index()['index'])

In [20]:
seqs = list(pep_spec.keys())

In [21]:
for idx in specs:
    print(seqs[idx])

AAADDGEEPK
AACEQLHQQQQQQQEETTAATLLLQGEEEGEED
AADIPGLK
AADKDTCFSTEGPNLVTR
AAEGVPK
AAEIDCQDIEER
AAESSAMAATEK
AAFEWNEEGAGSSPSPGLQPVR
AAFLNEVLR
AAFTMFSR
AAFVLPEFTR
AAHSEGHITAGLDMK
AAIAAAAAAAAAK
AAIAVPSAPYDITCLESFR
AALECLGQAEDLR
AALEELVR
AALLASR
AALNLLECGVSR
AALSGLLHR
AALSYFPR
AAPEESEAQAEGCSEER
AAPFTLEYR
AAPITQYLK
AAPLSLCALTAVDQSVLLLKPEAK
AAQHQANQAAVAAQAAAAAATVMTQSTAK
AAQTSDSEKIHLDEK
AASWLTHQGSFHGAFR
AATLNSR
AAVLGESAGLVR
AAVLHR
AAVSGLWGKVNADEVGGEALGR
AAYEHIYLVR
AAYLQENGKPLDEVLRK
AAYLQGLNSADLLK
ACEQPLGYICK
ACIPQSFTVDSSK
ACVDSNENGDLSK
ADAEVLR
ADAEVLRK
ADAMTLDGGTMFDAGKPPYK
ADAVQEATFQVELPR
ADAVQEATFQVELPRK
ADDASAATIEK
ADDELAALR
ADDILANPPR
ADDTFEALR
ADFAEITK
ADFHGGK
ADFQCFQQAR
ADINEENEETITK
ADLECTKPAA
ADLSGMSGSR
ADMPRPPETTTAVGAVVTAPHGR
ADMVIEAVFEDLGVK
ADPHLCDFLESHYLDKEVK
ADPSQDVR
ADSHFR
ADSSLDLSELAK
AEEDIDPNSVHSVPR
AEEINAEIER
AEETETAAPPPDKDR
AEGIPEFYDYDVALVK
AEGVNSKEDEELR
AEGVTALPYFLFK
AEILNLVK
AELLVAEAPSKPITVTVEEQR
AEMLQALASLEAVLLQTVYNTK
AENEALAMQK
AENQRPAEDSALSPGPLAGAK
AEPATPAEAAQSDR
AEQVDGVIN